In [258]:
import pandas as pd
import numpy as np

## Attempt 1

My intial code attempt. Quite inefficent as the dataframe is iterated over each time and a new slightly large mask created. Also, the entire text file needs to be stroed in memory, much better to loop over lines and store only those I's that don't have corresponding E's

In [313]:
times = np.array([1000,2000,2200,2400,2500,4000])
d = {'type':['I','I','I','E','E','E'],'id':[100,101,102,101,102,100],'price':[10.0,13.0,13.0,np.nan,np.nan,np.nan]}
test_data = pd.DataFrame(data=d,index=times)
test_data['high'] = np.nan
test_data

,id,price,type,high
1000,100,10,I,NaN
2000,101,13,I,NaN
2200,102,13,I,NaN
2400,101,NaN,E,NaN
2500,102,NaN,E,NaN
4000,100,NaN,E,NaN


In [311]:
for i in range(test_data.shape[0]):
    mask = test_data.iloc[:i+1,:].groupby('id')['id'].transform('count')==1
    
    print test_data.iloc[:i+1,:][mask]['price']
    print test_data.iloc[:i+1,:][mask]['price'].max()
    print 
    test_data['high'].iloc[i] = test_data.iloc[:i+1,:][mask]['price'].max()

delta_t = times[1:] - times[:-1]
np.nansum(delta_t*test_data.high.values[:-1])/sum(delta_t)
    

1000    10
Name: price, dtype: float64
10.0

1000    10
2000    13
Name: price, dtype: float64
13.0

1000    10
2000    13
2200    13
Name: price, dtype: float64
13.0

1000    10
2200    13
Name: price, dtype: float64
13.0

1000    10
Name: price, dtype: float64
10.0

Series([], Name: price, dtype: float64)
nan



10.5

## Attempt 2

In [ ]:
import os
import numpy as np

In [ ]:
class InsertEntry():
    def __init__(self, id_num, price):
        self.id_num = id_num
        self.price = price
    
    def __str__(self):
        return str([self.id_num, self.price])

In [ ]:
entry1 = InsertEntry(1000,'I',101,10)
entry2 = InsertEntry(2000,'I',102,13)
entry3 = InsertEntry(2200,'I',103,13)
entry4 = InsertEntry(2500,'E',101,None)

In [55]:
orderbook = OrderBook()

In [56]:
orderbook.add_entry(entry1)
orderbook.add_entry(entry2)
orderbook.add_entry(entry3)

In [57]:
orderbook.add_entry(entry4)

In [315]:
def is_non_zero_file(fpath):  
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

In [305]:
class OrderBook():
    def __init__(self):
        self.value = []
        self.high_price = None
        self.latest_time = 0
        self.delta_t = 0
        self.elapsed_time = 0
        self.time_scaled_high_price = 0
     
    def __str__(self):
         return str([str(i) for i in self.value])
        
    def insert(self,entry):
        self.value.append(entry)
    
    def erase(self,id_num):
        for i, o in enumerate(self.value):
            if o.id_num == id_num:
                del self.value[i]
                break

    def get_high_price(self):
        try:
            self.high_price = max([entry.price for entry in self.value if len(self.value)>0])
        except ValueError: # catches case when orderbook is empty, sets high value to NaN (as requested)
            self.high_price = np.nan
    
    def update_elapsed_time(self):
#         print 'before', self.elapsed_time
        self.elapsed_time = self.elapsed_time + self.delta_t
#         print 'afer', self.elapsed_time
        
    def update_time_scaled_high_price(self):
        self.time_scaled_high_price = self.time_scaled_high_price + self.high_price*self.delta_t
        
#     def get_positions(list, item):
#         return [(i, sublist.index(item)) for i, sublist in enumerate(list) if item in sublist]


In [325]:
if __name__ == "__main__":
    orderbook = OrderBook()
    file_path = '/Users/camcairns/Dropbox/py_modules/data_science_projects/OxAM/test_data.txt'
    if is_non_zero_file(file_path):
        with open(file_path) as f:
            for line in f:
                new_line = line.split()
                orderbook.delta_t = float(new_line[0]) - orderbook.latest_time
                orderbook.latest_time = float(new_line[0])
                orderbook.get_high_price()
                print orderbook.high_price
                if ~np.isnan(orderbook.high_price):
                    orderbook.update_elapsed_time()
                    orderbook.update_time_scaled_high_price()
                else:
                    pass
                if new_line[1] == 'I':
                    entry = InsertEntry(int(new_line[0]),str(new_line[1]),int(new_line[2]),float(new_line[3]))
                    orderbook.insert(entry)
                else:
                    orderbook.erase(int(new_line[2]))
            print 'The time-weighted high price is', orderbook.time_scaled_high_price/orderbook.elapsed_time
            print orderbook.time_scaled_high_price
            print orderbook.elapsed_time
    else:
        print 'ASCII text file is empty! No data to compute time weighted high price.'
        

[]
nan
["[1000, 'I', 100, 10.0]"]
10.0
["[1000, 'I', 100, 10.0]", "[2000, 'I', 101, 13.0]"]
13.0
["[1000, 'I', 100, 10.0]", "[2000, 'I', 101, 13.0]", "[2200, 'I', 102, 13.0]"]
13.0
["[1000, 'I', 100, 10.0]", "[2200, 'I', 102, 13.0]"]
13.0
["[1000, 'I', 100, 10.0]"]
10.0
[]
nan
["[5000, 'I', 103, 9.0]"]
9.0
The time-weighted high price is 9.9
49500.0
5000.0


In [360]:
import pandas as pd

In [510]:
df = pd.read_csv('/Users/camcairns/Dropbox/py_modules/data_science_projects/OxAM/generated_data_1million_orders.txt',sep=' ')

In [512]:
df.shape

(1999999, 4)

In [362]:
df.shape

(199999, 1)

In [516]:
(10*1000+13*500+10*1500+9*2000+10*1000.)/(1000+500+1500+2000+1000.)

9.916666666666666

In [526]:
not math.isnan(float('NaN'))

False

## Attempt 3

In [494]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [495]:
def is_non_zero_file(fpath):  
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

In [496]:
def unpack_new_line(line):
    if line.split()[1] == 'I':
        new_timestamp, new_flag, new_id_num, new_price = line.split()
        new_price = float(new_price)
    else:
        new_timestamp, new_flag, new_id_num = line.split()
        new_price = None
    new_timestamp = int(new_timestamp)
    new_id_num = int(new_id_num)
    return new_timestamp, new_flag, new_id_num, new_price

In [497]:
class OrderBook():
    def __init__(self):
        self.dictionary = {}
        self.high_price = None
        self.latest_time = 0
        self.delta_t = 0
        self.elapsed_time = 0
        self.time_scaled_high_price = 0
        self.recalculate_max_high_flag = True
     
    def __str__(self):
         return str(self.dictionary)
        
    def insert(self,id_num,price):
        self.recalculate_max_high_flag = True if (price > self.high_price) or (np.isnan(self.high_price)) else False
        self.dictionary[id_num] = price
    
    def erase(self,id_num):
        self.recalculate_max_high_flag = True if self.dictionary[id_num] == self.high_price else False
        del self.dictionary[id_num]

    def get_high_price(self):
        if self.recalculate_max_high_flag:
            try:
                self.high_price = max(self.dictionary.values())
            except ValueError: # catches case when orderbook is empty, sets high value to NaN (as requested)
                self.high_price = np.nan
        else:
            pass
    
    def update_elapsed_time(self):
        self.elapsed_time = self.elapsed_time + self.delta_t
        
    def update_time_scaled_high_price(self):
        self.time_scaled_high_price = self.time_scaled_high_price + self.high_price*self.delta_t

In [498]:
def find_time_average_high():
    orderbook = OrderBook()
    file_path = '/Users/camcairns/Dropbox/py_modules/data_science_projects/OxAM/generated_data_1million_orders.txt'
    if is_non_zero_file(file_path):
        with open(file_path) as f:
            for line in f:
                new_timestamp, new_flag, new_id_num, new_price = unpack_new_line(line)
                orderbook.delta_t = new_timestamp - orderbook.latest_time
                orderbook.latest_time = new_timestamp
                orderbook.get_high_price()
#                 print orderbook.high_price
                if np.isnan(orderbook.high_price):
                   pass
                else:
                    orderbook.update_elapsed_time()
                    orderbook.update_time_scaled_high_price()
                if new_flag == 'I':
                    orderbook.insert(new_id_num,new_price)
                else:
                    orderbook.erase(new_id_num)
            print 'The time-weighted high price is', orderbook.time_scaled_high_price/orderbook.elapsed_time
            print orderbook.time_scaled_high_price
            print orderbook.elapsed_time
    else:
        print 'ASCII text file is empty! No data to compute time weighted high price.'
        

In [499]:
%lprun -s -f find_time_average_high -T linebreak_results.txt find_time_average_high()

The time-weighted high price is 19.9996919594
1.00008264566e+11
5000490246

*** Profile printout saved to text file u'linebreak_results.txt'. 


# Attempt 4

In [500]:
import numpy as np
import os

def is_non_zero_file(fpath):  
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

class OrderBook():
    def __init__(self):
        self.ledger = {}
        self.high_price = None
        self.latest_time = 0
        self.delta_t = 0
        self.elapsed_time = 0
        self.time_scaled_high_price = 0
        self.recalculate_max_high_flag = True
     
    def __str__(self):
         return str(self.ledger)
        
    def insert(self,id_num,price):
        self.recalculate_max_high_flag = True if (price > self.high_price) or (np.isnan(self.high_price)) else False
        self.ledger[id_num] = price
    
    def erase(self,id_num):
        self.recalculate_max_high_flag = True if self.ledger[id_num] == self.high_price else False
        del self.ledger[id_num]

    def get_high_price(self):
        if self.recalculate_max_high_flag:
            try:
                self.high_price = max(self.ledger.values())
            except ValueError: # catches case when orderbook is empty, sets high value to NaN (as requested)
                self.high_price = np.nan
        else:
            pass
    
    def update_elapsed_time(self):
        self.elapsed_time = self.elapsed_time + self.delta_t
        
    def update_time_scaled_high_price(self):
        self.time_scaled_high_price = self.time_scaled_high_price + self.high_price*self.delta_t


In [501]:
def find_time_average_high():
    orderbook = OrderBook()
    file_path = '/Users/camcairns/Dropbox/py_modules/data_science_projects/OxAM/generated_data_1million_orders.txt'
    if is_non_zero_file(file_path):
        with open(file_path) as f:
            for line in f:
                new_line = line.split()
                orderbook.delta_t = float(new_line[0]) - orderbook.latest_time
                orderbook.latest_time = float(new_line[0])
                orderbook.get_high_price()
#                 print orderbook.high_price
                if np.isnan(orderbook.high_price):
                   pass
                else:
                    orderbook.update_elapsed_time()
                    orderbook.update_time_scaled_high_price()
                if new_line[1] == 'I':
                    orderbook.insert(int(new_line[2]),float(new_line[3]))
                else:
                    orderbook.erase(int(new_line[2]))
            print 'The time-weighted high price is', orderbook.time_scaled_high_price/orderbook.elapsed_time
            print orderbook.time_scaled_high_price
            print orderbook.elapsed_time
    else:
        print 'ASCII text file is empty! No data to compute time weighted high price.'


In [502]:
%lprun -s -f find_time_average_high -T no_linebreak_results.txt find_time_average_high()

The time-weighted high price is 19.9996919594
1.00008264566e+11
5000490246.0

*** Profile printout saved to text file u'no_linebreak_results.txt'. 


In [504]:
36/43.

0.8372093023255814

In [65]:
if A:
    print 'foo'
else:
    print 'bar'

foo


In [3]:
def get_positions(list, item):
    return [(i, sublist.index(item)) for i, sublist in enumerate(list) if item in sublist]

def get_max(list):
    return [sublist[3] for i, sublist in enumerate(list) if len(sublist)>0]

lst = []
new_time = 0
elapsed_time = 0
time_weighted_high = 0
with open('/Users/camcairns/Dropbox/py_modules/data_science_projects/OxAM/generated_data.txt') as f:
    for line in f:
        new_line = line.split()
        delta_t =  float(new_line[0]) - new_time
        new_time = float(new_line[0])
        prices = [float(i[3]) for i in lst]
        if prices:
            high = max(prices)
            elapsed_time = elapsed_time + delta_t
        else:
            high = 0 # handles cases where there are no orders in book (time does not contribute to elapsed time in this case)
        time_weighted_high = time_weighted_high + high*delta_t
        if new_line[1] == 'I':
            lst.append(new_line)
        elif new_line[1] == 'E':
            del lst[get_positions(lst,new_line[2])[0][0]]

    
    time_average = time_weighted_high/elapsed_time
    print time_average

(2641.0, 0)
(434.0, 17.486532275049022)
(1439.0, 17.486532275049022)
(4843.0, 18.931177707586937)
(4434.0, 18.931177707586937)
(711.0, 18.931177707586937)
(3707.0, 18.931177707586937)
(3003.0, 18.931177707586937)
(461.0, 18.931177707586937)
(2148.0, 18.931177707586937)
(2622.0, 18.931177707586937)
(4028.0, 18.931177707586937)
(3802.0, 18.931177707586937)
(1873.0, 18.931177707586937)
(3046.0, 18.931177707586937)
(406.0, 18.931177707586937)
(2777.0, 18.931177707586937)
(1028.0, 18.931177707586937)
(1730.0, 18.931177707586937)
(4957.0, 18.931177707586937)
(4427.0, 19.39753830186035)
(3966.0, 19.39753830186035)
(1959.0, 19.39753830186035)
(612.0, 19.39753830186035)
(4013.0, 19.39753830186035)
(2471.0, 19.39753830186035)
(1180.0, 19.39753830186035)
(4629.0, 19.39753830186035)
(170.0, 19.58127923069857)
(4317.0, 19.58127923069857)
(661.0, 19.58127923069857)
(2103.0, 19.58127923069857)
(1937.0, 19.58127923069857)
(2372.0, 19.58127923069857)
(2270.0, 19.58127923069857)
(965.0, 19.5812792306985

In [345]:
class SomeClass:
    def __init__(self):
        self.xs = [['1','2','4','6'], ['7','0','1','4']]

    def get_positions(self, target):
        return [(i,e.index(target)) for i,e in enumerate(self.xs)]

In [348]:
model = SomeClass()
model.get_positions('1')    # returns [(0,0), (1,2)]

[(0, 0), (1, 2)]

Problem 1: Write an iterator class reverse_iter, that takes a list and iterates it from the reverse direction. ::

In [72]:
class reverse_yrange:
    def __init__(self, n):
        self.i = n
        self.n = 0

    def __iter__(self):
        return self

    def next(self):
        if self.i >= self.n:
            i = self.i
            self.i -= 1
            return i
        else:
            raise StopIteration()

In [73]:
y = reverse_yrange(10)
list(y)

In [75]:
def yrange(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [77]:
y =yrange(10)

In [79]:
list(y)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [81]:
def integers():
    """Infinite sequence of integers."""
    i = 1
    while True:
        yield i
        i = i + 1

def squares():
    for i in integers():
        yield i * i

def take(n, seq):
    """Returns first n values from the given sequence."""
    seq = iter(seq)
    result = []
    try:
        for i in range(n):
            result.append(seq.next())
    except StopIteration:
        pass
    return result

In [87]:
print take(5, squares()) # prints [1, 4, 9, 16, 25]

[1, 4, 9, 16, 25]


In [90]:
pyt = ((x, y, z) for z in integers() for y in xrange(1, z) for x in range(1, y) if x*x + y*y == z*z)
take(10, pyt)

[(3, 4, 5),
 (6, 8, 10),
 (5, 12, 13),
 (9, 12, 15),
 (8, 15, 17),
 (12, 16, 20),
 (15, 20, 25),
 (7, 24, 25),
 (10, 24, 26),
 (20, 21, 29)]

Problem 2: Write a program that takes one or more filenames as arguments and prints all the lines which are longer than 40 characters.

In [109]:
import glob

def readfiles(filenames):
    for f in filenames:
        for line in open(f):
            yield line

def grep(pattern, lines):
    return (line for line in lines if pattern in lines)

def line_len(n, lines):
    return (line for line in lines if len(line)<n)

def printlines(lines):
    for line in lines:
        print line,

def main(n, filenames):
    lines = readfiles(filenames)
    lines = line_len(n, lines)
    printlines(lines)

In [111]:
filenames = glob.glob('/Users/camcairns/Documents/war_poems/*.txt')

In [112]:
main(40,filenames)

In Flanders Fields, by John McRae

In Flanders fields the poppies blow
Between the crosses, row on row,
That mark our place; and in the sky
The larks, still bravely singing, fly
Scarce heard amid the guns below.

We are the Dead. Short days ago
We lived, felt dawn, saw sunset glow,
Loved and were loved, and now we lie
In Flanders fields.

Take up our quarrel with the foe:
To you from failing hands we throw
The torch; be yours to hold it high.
If ye break faith with us who die
In Flanders fields. Marching Men, by Marjorie Pickthall

Under the level winter sky
I saw a thousand Christs go by.
They sang an idle song and free
As they went up to calvary.

Careless of eye and coarse of lip,
They marched in holiest fellowship.

They supped the sacrament of death.
And for each one, far off, apart,
Seven swords have rent a woman's heart. The Cenotaph, by Charlotte Mew

Speaking so wistfully of other Springs
To lovers - to mothers
Here, too, lies he:
Under the purple, the green, the red,
Only, wh

Problem 3: Write a function findfiles that recursively descends the directory tree for the specified directory and generates paths of all the files in the tree.

In [114]:
import os
import sys

rootdir = sys.argv[1]

In [121]:
import os
from os.path import join, getsize
for root, dirs, files in os.walk('/Users/camcairns/Documents/war_poems/'):
    print root, "consumes"
    print dirs
    print files
#     print sum(getsize(join(root, name)) for name in files),
#     print "bytes in", len(files), "non-directory files"
#     if 'CVS' in dirs:
#         dirs.remove('CVS')  # don't visit CVS directories

/Users/camcairns/Documents/war_poems/ consumes
[]
['.DS_Store', 'In_Flanders_Fields_John_McRae.txt', 'Marching_Men_Marjorie_Pickthall.txt', 'The_Cenotaph_Charlotte_Mew.txt', 'The_Soldier_Rupert_Brooke.txt']
